In [1]:
from __future__ import division
import numpy as np

from pyspark import SparkConf
from pyspark import SparkContext

In [2]:
conf = SparkConf()
conf.setMaster('spark://ip-172-31-9-200:7077')
conf.setAppName('spark_analytics_chpt_4')
conf.set("spark.executor.memory", "10g")
sc = SparkContext(conf=conf)

Data from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/

In [3]:
raw_data = sc.textFile('covtype.data')

In [4]:
raw_data.count()

581012

In [5]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

def to_float(s):
    try:
        return float(s)
    except:
        return float('nan')
    
def clean(line):
    for x in line.split(','):
        values = map(to_float, line.split(','))
        featureVector = Vectors.dense(values[:-1])
        label = values[-1] - 1
        return LabeledPoint(label, featureVector)

In [6]:
data = raw_data.map(clean)

In [7]:
data.take(5)

[LabeledPoint(4.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(4.0, [2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(1.0, [2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(1.0, [2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.

In [8]:
training_data, cv_data, test_data = data.randomSplit([0.8, 0.1, 0.1])

In [9]:
training_data.cache()
cv_data.cache()
test_data.cache()

PythonRDD[6] at RDD at PythonRDD.scala:43

In [10]:
training_data.count(), cv_data.count(), test_data.count()

(464818, 58325, 57869)

## Decision Tree

In [11]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

In [12]:
model = DecisionTree.trainClassifier(training_data, 7, {}, 'gini', 4, 100)

In [13]:
predictions = model.predict(data.map(lambda x: x.features))
labels_and_predictions = data.map(lambda lp: lp.label).zip(predictions)

In [14]:
metrics = MulticlassMetrics(labels_and_predictions)

In [15]:
metrics.confusionMatrix()

DenseMatrix(6, 6, [143414.0, 64794.0, 108.0, 10.0, 0.0, 3514.0, 56061.0, 222001.0, ..., 123.0, 0.0, 11315.0, 310.0, 0.0, 0.0, 0.0, 8885.0], 0)

In [16]:
metrics.precision()

0.6997566315325673

In [18]:
map(lambda cat: (metrics.precision(cat), metrics.recall(cat)), [0, 1, 2, 3, 4, 6])

[(0.6769920694864048, 0.6803643436595664),
 (0.7836223663170974, 0.729323604682105),
 (0.8678749230855289, 0.6241576988836367),
 (0.4055333090644339, 0.3523086654016445),
 (0.012956915622037291, 0.7109826589595376),
 (0.4332033154558752, 0.6952813209171297)]

In [19]:
def classProbabilities(data):
    countsByCategory = data.map(lambda x: x.label).countByValue()
    counts = np.array(countsByCategory.values()) / sum(countsByCategory.values())
    return counts

In [20]:
trainPriorProbabilities = classProbabilities(training_data)

In [21]:
cvPriorProbabilities = classProbabilities(cv_data)

In [22]:
sum([x[0] * x[1] for x in zip(trainPriorProbabilities, cvPriorProbabilities)])

0.37673722119246689

In [23]:
for impurity in ('gini', 'entropy'):
    for depth in (1, 20):
        for bins in (10, 300):
            model = DecisionTree.trainClassifier(training_data, 7, {}, impurity, depth, bins)
            predictions = model.predict(cv_data.map(lambda x: x.features))
            labels_and_predictions = cv_data.map(lambda lp: lp.label).zip(predictions)
            metrics = MulticlassMetrics(labels_and_predictions)
            accuracy = metrics.precision()
            print (impurity, depth, bins), accuracy

('gini', 1, 10) 0.632353193313
('gini', 1, 300) 0.631290184312
('gini', 20, 10) 0.891350192885
('gini', 20, 300) 0.905940848693
('entropy', 1, 10) 0.487509644235
('entropy', 1, 300) 0.487509644235
('entropy', 20, 10) 0.895207886841
('entropy', 20, 300) 0.914582083155


In [24]:
model = DecisionTree.trainClassifier(training_data.union(cv_data), 7, {}, 'entropy', 20, 300)

In [25]:
predictions = model.predict(data.map(lambda x: x.features))
labels_and_predictions = data.map(lambda lp: lp.label).zip(predictions)
metrics = MulticlassMetrics(labels_and_predictions)
accuracy = metrics.precision()
print accuracy

0.941133745947


## Random Forest

In [26]:
from pyspark.mllib.tree import RandomForest

In [27]:
forest = RandomForest.trainClassifier(training_data, 7, {10:4, 11:40}, 20, 'auto', 'entropy', 30, 300)

In [28]:
predictions = model.predict(data.map(lambda x: x.features))
labels_and_predictions = data.map(lambda lp: lp.label).zip(predictions)
metrics = MulticlassMetrics(labels_and_predictions)
accuracy = metrics.precision()
print accuracy

0.941133745947


In [29]:
from pyspark.mllib.linalg import Vectors

input = '2709,125,28,67,23,3224,253,207,61,6094,0,29'
vector = Vectors.dense([to_float(x) for x in input.split(',')])

In [30]:
result = forest.predict(vector)

Py4JJavaError: An error occurred while calling o725.predict.
: java.lang.ArrayIndexOutOfBoundsException: 35
	at org.apache.spark.mllib.linalg.DenseVector.apply(Vectors.scala:560)
	at org.apache.spark.mllib.tree.model.Node.predict(Node.scala:91)
	at org.apache.spark.mllib.tree.model.DecisionTreeModel.predict(DecisionTreeModel.scala:58)
	at org.apache.spark.mllib.tree.model.TreeEnsembleModel$$anonfun$predictByVoting$1.apply(treeEnsembleModels.scala:311)
	at org.apache.spark.mllib.tree.model.TreeEnsembleModel$$anonfun$predictByVoting$1.apply(treeEnsembleModels.scala:310)
	at scala.collection.Iterator$class.foreach(Iterator.scala:727)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1157)
	at scala.collection.IterableViewLike$Transformed$class.foreach(IterableViewLike.scala:42)
	at scala.collection.SeqViewLike$AbstractTransformed.foreach(SeqViewLike.scala:43)
	at org.apache.spark.mllib.tree.model.TreeEnsembleModel.predictByVoting(treeEnsembleModels.scala:310)
	at org.apache.spa